**TASK** Recap:

- Jak działa perceptron?
- Jaka jest podstawowa operacja podczas propagacji informacji w przód w Sieci Feed Forward?
- Po co są funkcje aktywacji w sieciach neuronowych?
- Jakie są ograniczenia uczenia SSN metodą Stochastycznego Spadku wzdłuż gradientu?
- Co to są warsty Dropout i BatchNormalization i po co się je stosuje w sieciach neuronowych?
- Czym charakteryzują się neuronowe sieci konwolucyjne?
- Co to jest operacja splotu?
- Czym charakteryzują się neuronowe sieci rekurencyjne?
- Jakie istnieją metody poprawy skuteczności modeli rekurencyjnych w przetwarzaniu języka naturalnego?

<style>
td {
  text-align: center;
}

th {
  text-align: center;
}
</style>

## What are GANs?
[Generative Adversarial Networks](https://arxiv.org/abs/1406.2661) (GANs) are one of the most interesting ideas in computer science today. Two models are trained simultaneously by an adversarial process. A *generator* ("the artist") learns to create images that look real, while a *discriminator* ("the art critic") learns to tell real images apart from fakes.

![A diagram of a generator and discriminator](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/gan1.png?raw=1)

During training, the *generator* progressively becomes better at creating images that look real, while the *discriminator* becomes better at telling them apart. The process reaches equilibrium when the *discriminator* can no longer distinguish real images from fakes.

![A second diagram of a generator and discriminator](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/gan2.png?raw=1)



In [ ]:
# To generate GIFs
!pip install imageio
!pip install git+https://github.com/tensorflow/docs

In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
from functools import partial
import numpy as np
import os
import PIL
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import time
import tensorflow as tf
import gdown
from zipfile import ZipFile

from IPython import display

## Generacja twarzy z użyciem sieci GAN i danych celeb.

Pobieranie danych. To jest duży zbiór danych, nie będziemy używać całego zbioru do treningu bo nie mamy wystarczających zasobów.

Do póżniejszej części zadania można usunąć ten plik z pamięci.

In [ ]:
os.makedirs("celeba_gan")

url = "https://drive.google.com/uc?id=1O7m1010EJjLE5QxLZiM9Fpjs7Oj6e684"
output = "celeba_gan/data.zip"
gdown.download(url, output, quiet=True)

with ZipFile("celeba_gan/data.zip", "r") as zipobj:
    zipobj.extractall("celeba_gan")


Stworzenie obiektu Dataset do uczenia w Tensorflow.

In [ ]:
dataset = keras.utils.image_dataset_from_directory(
    "celeba_gan", label_mode=None, image_size=(64, 64), batch_size=64
)
dataset = dataset.map(lambda x: x / 255.0)


Stworzenie próbki danych treningowych jako część oryginalnego zbioru danych.

In [ ]:
dim = lambda x: x[:640, ...]

train_dataset = dataset.map(dim)

del dataset

Wizualizacja przykładowej twarzy z zbioru danych

In [ ]:
for x in train_dataset:
    plt.axis("off")
    plt.imshow((x.numpy() * 255).astype("int32")[0])
    break


Model generatora: bierze jako wejście wektor wartości rzeczywistych (random noise) i tworzy obraz używając kombinacji warst liniowych i konwolucyjnych.

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(16*16*128, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((16, 16, 128)))
    assert model.output_shape == (None, 16, 16, 128)  # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 32, 32, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 64, 64, 3)

    return model

Przykładowy obraz z generatora przed treningiem.

In [ ]:
generator = make_generator_model()

noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0, :, :, 0])

Dyskryminator: bierze jako wejście obraz i ocenia czy jest wygenerowany czy oryginalny - jedno wyjście.

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[64,64, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

In [ ]:
discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print (decision)

#### Poniższe komórki definiują funkcję straty dla modeli generatora i dyskryminatora

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam()
discriminator_optimizer = tf.keras.optimizers.Adam()

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
# Ustawienie parametrów treningu
EPOCHS = 5
noise_dim = 100
num_examples_to_generate = 8
BATCH_SIZE = 64

# ustawienie globalnego ziarna dla wszystkich losowych generacji
seed = tf.random.normal([num_examples_to_generate, noise_dim])

Defninicją pojedynczego kroku treningu modelu GAN:
- stworzenie szumu
- definicja obliczania gradientu z obiektem GradientTape
- generacja obrazu
- wywołanie dyskryminatora na obrazie generowanym i orginalnym
- wyznaczenie straty dla generatora i dyskryminatora
- obliczenie i aplikacja gradientów do wag sieci

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

Definicja treningu modelu GAN

In [ ]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)

    # Save the model every 15 epochs
    if (epoch + 1) % 5 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)
  generator.save("final_model.keras")


Funkcja pomocnicza do generacji obrazów podczas treningu

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4, 4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5)
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

Trening sieci GAN. Może potrwać około 15 minut.

In [ ]:
train(train_dataset, EPOCHS)

Generacja przykładowego obrazu po treningu.

In [ ]:
test_input = tf.random.normal([1, 100])
generated_image_after_training = generator(test_input, training=False)

In [ ]:
plt.imshow(generated_image_after_training[0, :, :, 0])

Load the saved model

In [ ]:
model = keras.models.load_model("final_model.keras")

**TASK** Zbudować sieć GAN dla danych fashion mnist:
- wczytać i przygotować dane do treningu
- zdefiniowac strukturę sieci GAN - dyksryminator i generator. **NOTE** Istotne jest dobranie odpowiednich kształtów poszczególnych warstw sieci, tak żeby odpowiadały kształtowi obrazków wejściowych (x,y, liczba kanałów)
- dobrać odpowiednią wielkość BATCH'a
- wytrenować model GAN
- wygenerować 5 obrazów z losowo wytworzonych danych wejściowych (random noise)

In [ ]:
(train_images, train_labels), (_, _) = tf.keras.datasets.fashion_mnist.load_data()

**TASK** Uzupełnić poprawnie poniższą komórke do przygotowania danych do uczenia

In [ ]:
train_images = train_images.reshape(ilość danych, rozmiar obrazka oś x, rozmiar obrazka oś y, ilość kanałow).astype(zmiana na typ float32)

In [ ]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256
train_images = # Znormalizować obrazy do zakresu [0, 1]

Stworzenie obiektu Dataset do treningu sieci GAN

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
